Классификация текста

In [0]:
import torch
import random
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
SEED = 0

In [0]:
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
!pip install transformers

     |████████████████████████████████| 645kB 6.1MB/s 
     |████████████████████████████████| 890kB 16.2MB/s 
     |████████████████████████████████| 1.1MB 32.2MB/s 
     |████████████████████████████████| 3.8MB 36.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=43b048a04ffcb59a52f2cecf47388afe7a20ab8584dbcc2d26aeec54609027ef
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [0]:
from transformers import BertTokenizer

In [7]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [8]:
len(tokenizer.vocab)

30522

In [0]:
print(tokenizer.tokenize("I create my BERT model"))

['i', 'create', 'my', 'bert', 'model']


In [0]:
print(tokenizer.convert_tokens_to_ids(tokenizer.tokenize("I create my BERT model")))

[1045, 3443, 2026, 14324, 2944]


In [0]:
CLS = tokenizer.cls_token_id
SEP = tokenizer.sep_token_id
PAD = tokenizer.pad_token_id
UNK = tokenizer.unk_token_id

In [0]:
print(tokenizer.max_model_input_sizes)

{'bert-base-uncased': 512, 'bert-large-uncased': 512, 'bert-base-cased': 512, 'bert-large-cased': 512, 'bert-base-multilingual-uncased': 512, 'bert-base-multilingual-cased': 512, 'bert-base-chinese': 512, 'bert-base-german-cased': 512, 'bert-large-uncased-whole-word-masking': 512, 'bert-large-cased-whole-word-masking': 512, 'bert-large-uncased-whole-word-masking-finetuned-squad': 512, 'bert-large-cased-whole-word-masking-finetuned-squad': 512, 'bert-base-cased-finetuned-mrpc': 512, 'bert-base-german-dbmdz-cased': 512, 'bert-base-german-dbmdz-uncased': 512, 'bert-base-finnish-cased-v1': 512, 'bert-base-finnish-uncased-v1': 512, 'bert-base-dutch-cased': 512}


In [0]:
def sent2toks(sent):
  toks = tokenizer.tokenize(sent)
  toks = toks[:tokenizer.max_model_input_sizes['bert-base-uncased'] - 2]
  return toks

In [0]:
sent2toks('I create my BERT model')

['i', 'create', 'my', 'bert', 'model']

In [11]:
!pip install torchtext

In [0]:
from torchtext import data
from torchtext import datasets

In [0]:
TEXT = data.Field(batch_first=True, use_vocab=False, tokenize=sent2toks, preprocessing=tokenizer.convert_tokens_to_ids, init_token=CLS, eos_token=SEP, pad_token=PAD, unk_token=UNK)
LABEL = data.LabelField(dtype = torch.float)

In [14]:
data_train, data_test = datasets.TREC.splits(TEXT, LABEL)

downloading train_5500.label


train_5500.label: 100%|██████████| 336k/336k [00:00<00:00, 871kB/s]


downloading TREC_10.label


TREC_10.label: 100%|██████████| 23.4k/23.4k [00:00<00:00, 298kB/s]


In [32]:
print(len(data_train), len(data_test))
print(tokenizer.convert_ids_to_tokens(vars(data_train.examples[17])['text']))
vars(data_train.examples[17])['label']

5452 500
['how', 'many', 'jews', 'were', 'executed', 'in', 'concentration', 'camps', 'during', 'wwii', '?']


'NUM'

In [0]:
example = data_train.examples[0]
print(example)
print(vars(example))

{'text': [2129, 2106, 14262, 2546, 9527, 4503, 1999, 1998, 2059, 2681, 3607, 1029], 'label': 'DESC'}


In [33]:
LABEL.build_vocab(data_train)
print(LABEL.vocab.stoi)

defaultdict(<function _default_unk_index at 0x7f55f8d958c8>, {'ENTY': 0, 'HUM': 1, 'DESC': 2, 'NUM': 3, 'LOC': 4, 'ABBR': 5})


In [0]:
BATCH_SIZE = 32
train_it, test_it = data.BucketIterator.splits((data_train, data_test), batch_size=BATCH_SIZE, device=device)

In [0]:
lbl = torch.zeros((len(batch_0.label), len(LABEL.vocab)))
for i in range(batch_0.label.shape[0]):
  lbl[i][int(batch_0.label[i])] = 1
print(batch_0.label, lbl)

tensor([1., 2., 0., 0., 1., 0., 4., 4., 2., 0., 4., 0., 3., 0., 4., 2., 0., 3.,
        3., 1., 0., 2., 1., 0., 1., 1., 3., 1., 0., 0., 2., 1.],
       device='cuda:0') tensor([[0., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1., 0.],
        [0., 0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0.],
        [1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0.],
        [0., 0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [0., 1.

In [0]:
from transformers import BertModel

In [44]:
bert = BertModel.from_pretrained('bert-base-uncased')

In [45]:
bert

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [0]:
import torch.nn as nn
import torch.nn.functional as func

In [0]:
class Classifier(nn.Module):
  def __init__(self, bert, hid_dim, out_dim, num_lay, bi, drop):
    super().__init__()

    self.inp2emb = bert
    self.emb2hid = nn.GRU(bert.config.to_dict()['hidden_size'], hid_dim, num_layers=num_lay, bidirectional=bi, batch_first=True, dropout=drop)
    self.hid2out = nn.Linear(hid_dim * 2 if bi else hid_dim, out_dim)
    self.drop = nn.Dropout(drop)

  def forward(self, inp):
    with torch.no_grad():
      emb = self.inp2emb(inp)[0]
    
    hid, h = self.emb2hid(emb)
    
    if self.emb2hid.bidirectional:
      h = self.drop(torch.cat((h[-2,:,:], h[-1,:,:]), dim=1))
    else:
      h = self.drop(h[-1,:,:])
    
    out = self.hid2out(h)
    out = func.softmax(out, dim = 1)
    return out

In [0]:
HID_DIM = 256
OUT_DIM = len(LABEL.vocab)
NUM_LAY = 2
BI = True
DROP = 0.25

classifier = Classifier(bert, HID_DIM, OUT_DIM, NUM_LAY, BI, DROP)

In [0]:
sum(p.numel() for p in classifier.parameters() if p.requires_grad)

112243974

In [0]:
for name, param in classifier.named_parameters():
  if name.startswith('inp2emb'):
    param.requires_grad = False

In [0]:
sum(p.numel() for p in classifier.parameters() if p.requires_grad)

2761734

In [0]:
for name, param in classifier.named_parameters():
  if param.requires_grad:
    print(name)

emb2hid.weight_ih_l0
emb2hid.weight_hh_l0
emb2hid.bias_ih_l0
emb2hid.bias_hh_l0
emb2hid.weight_ih_l0_reverse
emb2hid.weight_hh_l0_reverse
emb2hid.bias_ih_l0_reverse
emb2hid.bias_hh_l0_reverse
emb2hid.weight_ih_l1
emb2hid.weight_hh_l1
emb2hid.bias_ih_l1
emb2hid.bias_hh_l1
emb2hid.weight_ih_l1_reverse
emb2hid.weight_hh_l1_reverse
emb2hid.bias_ih_l1_reverse
emb2hid.bias_hh_l1_reverse
hid2out.weight
hid2out.bias


In [0]:
import torch.optim as optim

In [0]:
classifier = classifier.to(device)

In [0]:
opt = optim.Adam(classifier.parameters())
crit = nn.BCELoss().to(device)

In [0]:
def accuracy(out, trg):
  ind = out.argmax(dim = 1)
  correct = (ind == trg).float()
  return correct.sum() / len(correct)

In [0]:
def lbl2trg(lbl):
  trg = torch.zeros((len(lbl), len(LABEL.vocab))).to(device)
  for i in range(len(lbl)):
    trg[i][int(lbl[i])] = 1
  return trg

In [0]:
def train(classifier, it, opt, crit):
  epoch_loss = 0
  epoch_acc = 0

  classifier.train()

  for batch in it:
    opt.zero_grad()
    out = classifier(batch.text).squeeze(1)
    
    loss = crit(out, lbl2trg(batch.label))
    acc = accuracy(out, batch.label)
    
    loss.backward()
    opt.step()
    
    epoch_loss += loss.item()
    epoch_acc += acc.item()
  
  return epoch_loss / len(it), epoch_acc / len(it)

In [0]:
def eval(classifier, it, crit):
  epoch_loss = 0
  epoch_acc = 0

  classifier.eval()

  with torch.no_grad():
    for batch in it:
      out = classifier(batch.text).squeeze(1)
      
      loss = crit(out, lbl2trg(batch.label))
      acc = accuracy(out, batch.label)

      epoch_loss += loss.item()
      epoch_acc += acc.item()
  
  return epoch_loss / len(it), epoch_acc / len(it)

In [0]:
import time

In [0]:
def epoch_time(start, end):
  delta = end - start
  delta_min = int(delta / 60)
  delta_sec = int(delta - delta_min * 60)
  return delta_min, delta_sec

In [0]:
best = float('inf')

In [0]:
EPOCH_NUM = 5

In [0]:
for i in range(EPOCH_NUM):
  start = time.time()

  train_loss, train_acc = train(classifier, train_it, opt, crit)
  test_loss, test_acc = eval(classifier, test_it, crit)

  end = time.time()
  mins, secs = epoch_time(start, end)

  if test_loss < best:
    best = test_loss
    torch.save(classifier.state_dict(), '/content/gdrive/My Drive/Models/classifier.pt')

  print('Epoch: ' + str(i))
  print('Time: ' + str(mins) + ' m ' + str(secs) + ' s')
  print('Train (loss | acc): ' + str(train_loss) + ' | ' + str(train_acc))
  print('Test (loss | acc): ' + str(test_loss) + ' | ' + str(test_acc))

Epoch: 0
Time: 0 m 7 s
Train (loss | acc): 0.09419661430282551 | 0.8896198830409356
Test (loss | acc): 0.08775715623050928 | 0.9058593772351742
Epoch: 1
Time: 0 m 7 s
Train (loss | acc): 0.07027911709632441 | 0.9214790449504964
Test (loss | acc): 0.06629626150242984 | 0.9441406242549419
Epoch: 2
Time: 0 m 7 s
Train (loss | acc): 0.052434892388202294 | 0.9449926900584795
Test (loss | acc): 0.08070120518095791 | 0.9117187522351742
Epoch: 3
Time: 0 m 7 s
Train (loss | acc): 0.04798082760344559 | 0.9449926900584795
Test (loss | acc): 0.06648045621113852 | 0.9378906264901161
Epoch: 4
Time: 0 m 7 s
Train (loss | acc): 0.03809517636426795 | 0.9554093567251462
Test (loss | acc): 0.0888174141291529 | 0.9164062514901161


In [0]:
best

0.06629626150242984

In [0]:
for i in range(EPOCH_NUM):
  start = time.time()

  train_loss, train_acc = train(classifier, train_it, opt, crit)
  test_loss, test_acc = eval(classifier, test_it, crit)

  end = time.time()
  mins, secs = epoch_time(start, end)

  if test_loss < best:
    best = test_loss
    torch.save(classifier.state_dict(), '/content/gdrive/My Drive/Models/classifier.pt')

  print('Epoch: ' + str(i))
  print('Time: ' + str(mins) + ' m ' + str(secs) + ' s')
  print('Train (loss | acc): ' + str(train_loss) + ' | ' + str(train_acc))
  print('Test (loss | acc): ' + str(test_loss) + ' | ' + str(test_acc))

Epoch: 0
Time: 0 m 7 s
Train (loss | acc): 0.034930355234169645 | 0.9594298245614035
Test (loss | acc): 0.06555778410984203 | 0.9449218772351742
Epoch: 1
Time: 0 m 7 s
Train (loss | acc): 0.024304803596994683 | 0.9747807017543859
Test (loss | acc): 0.06700775225181133 | 0.9449218772351742
Epoch: 2
Time: 0 m 7 s
Train (loss | acc): 0.021695025050427155 | 0.9758771929824561
Test (loss | acc): 0.07639386679511517 | 0.9324218742549419
Epoch: 3
Time: 0 m 7 s
Train (loss | acc): 0.01848218627719955 | 0.9791666666666666
Test (loss | acc): 0.11910981964319944 | 0.8960937522351742
Epoch: 4
Time: 0 m 7 s
Train (loss | acc): 0.015495714131907467 | 0.985014619883041
Test (loss | acc): 0.07790230696264189 | 0.9441406242549419


In [0]:
best

0.06555778410984203

In [0]:
for i in range(EPOCH_NUM):
  start = time.time()

  train_loss, train_acc = train(classifier, train_it, opt, crit)
  test_loss, test_acc = eval(classifier, test_it, crit)

  end = time.time()
  mins, secs = epoch_time(start, end)

  if test_loss < best:
    best = test_loss
    torch.save(classifier.state_dict(), '/content/gdrive/My Drive/Models/classifier.pt')

  print('Epoch: ' + str(i))
  print('Time: ' + str(mins) + ' m ' + str(secs) + ' s')
  print('Train (loss | acc): ' + str(train_loss) + ' | ' + str(train_acc))
  print('Test (loss | acc): ' + str(test_loss) + ' | ' + str(test_acc))

Epoch: 0
Time: 0 m 7 s
Train (loss | acc): 0.01616475453354522 | 0.9819078947368421
Test (loss | acc): 0.0957848101388663 | 0.9167968742549419
Epoch: 1
Time: 0 m 7 s
Train (loss | acc): 0.014181085221169173 | 0.9849537039360805
Test (loss | acc): 0.06962842981738504 | 0.9421874992549419
Epoch: 2
Time: 0 m 7 s
Train (loss | acc): 0.013237697717717342 | 0.9864766081871345
Test (loss | acc): 0.0818764518480748 | 0.9324218742549419
Epoch: 3
Time: 0 m 7 s
Train (loss | acc): 0.011692806343957836 | 0.9890350877192983
Test (loss | acc): 0.07940432186296675 | 0.9390625022351742
Epoch: 4
Time: 0 m 7 s
Train (loss | acc): 0.012847327716680576 | 0.9853801169590644
Test (loss | acc): 0.06762367824558169 | 0.9339843764901161


In [52]:
classifier.load_state_dict(torch.load('/content/gdrive/My Drive/Models/classifier.pt'))

<All keys matched successfully>

In [0]:
test_loss, test_acc = eval(classifier, test_it, crit)

In [59]:
print(test_loss, test_acc)

0.06555778381880373 0.9449218772351742


In [0]:
def classify(classifier, tokenizer, sent):
  classifier.eval()
  toks = tokenizer.tokenize(sent)[:tokenizer.max_model_input_sizes['bert-base-uncased'] - 2]
  inds = [CLS] + tokenizer.convert_tokens_to_ids(toks) + [SEP]
  inp = torch.LongTensor(inds).unsqueeze(0).to(device)
  return LABEL.vocab.itos[classifier(inp).argmax(dim = 1).item()]

In [0]:
def classify_list(classifier, tokenizer, sents):
  outs = []
  for sent in sents:
    outs.append(classify(classifier, tokenizer, sent))
  return outs

In [85]:
classify(classifier, tokenizer, "What are you thinking about?")

'DESC'

In [72]:
classify(classifier, tokenizer, "what does NLP stand for?")

'ABBR'

In [101]:
classify(classifier, tokenizer, "Do you know the address?")

'LOC'

In [74]:
classify_list(classifier, tokenizer, ["What does it mean?", "What kind of food do you prefer?", "What place is your favorite one?", "Where are you running?", "Whom did you give your phone number?", "How many books did you read?"])

['DESC', 'ENTY', 'LOC', 'LOC', 'HUM', 'NUM']

In [84]:
classify(classifier, tokenizer, "")

'ENTY'

In [80]:
classify_list(classifier, tokenizer, ["What is your favorite dish?", "What kind of food do you prefer?", "What place is your favorite one?", "Where are you running?", "Whom did you give your phone number?", "How many books did you read?"])

['ENTY', 'ENTY', 'LOC', 'LOC', 'HUM', 'NUM']